# Chain模块代码实战

In [ ]:
## 导入依赖库

!pip install langchain

## 1. LLM

LLMChain: 简单的LLM链
使用LangChain表达式语言来 实现最常见的链式操作类型

在任何LLM应用中，最常见的链接方式是将提示模板与LLM和可选的输出解析器组合起来。

In [1]:
from langchain.prompts import PromptTemplate
#from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI

from langchain.schema import StrOutputParser



prompt = PromptTemplate.from_template(
    "What is a good name for a company that makes {product}?"
)

chat_model = ChatOpenAI(model_name="gpt-3.5-turbo-0125")
## LCEL chain表达式语言， 利用管道符“|” 实现 链式调用
runnable = prompt | chat_model | StrOutputParser()

result = runnable.invoke({"product": "colorful socks"})
print(result)

"Rainbow Footwear Co."


以前的API（不推荐使用，但是建议了解一下）

In [2]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
#from langchain_openai import OpenAI
from langchain.chains import LLMChain

prompt_template = "What is a good name for a company that makes {product}?"

llm = OpenAI(model_name="gpt-3.5-turbo")

llm_chain = LLMChain(llm=llm, prompt=PromptTemplate.from_template(prompt_template))

result = llm_chain("colorful socks")
print(result)

{'product': 'colorful socks', 'text': 'Rainbow Threads'}


## 2.Sequential

SequentialChain: 串联式调用语言模型链 （将多条链路组合串起来使用）
在调用语言模型之后，下一步是对语言模型进行一系列的调用。当您希望将一次调用的输出作为另一次调用的输入时，这是特别有用的。
比如：
第一次调用语言模型（prompmt1|model|parse1)
第二次调用语言模型（prompmt2|model|parse2)
第一次的parse1，作为第二次的prompmt2输入内容

示例: 假设我们想创建一个链，首先创建剧情简介，然后根据简介生成一篇剧评。

In [3]:
from langchain.prompts import PromptTemplate

synopsis_prompt = PromptTemplate.from_template(
    """你是一位剧作家。给定一个剧目的标题，你的任务是为这个标题写一个剧情简介。

标题: {title}
剧作家: 这是上述剧目的剧情简介:"""
)

review_prompt = PromptTemplate.from_template(
    """您是《纽约时报》的戏剧评论家。根据剧情简介，您的工作是为该剧撰写一篇评论。.

剧情简介:
{synopsis}
上述剧目的《纽约时报》剧评家的评论:"""
)

In [8]:
from langchain_openai import ChatOpenAI
from langchain.schema import StrOutputParser

#llm = ChatOpenAI(model_name="gpt-4")
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125")

chain = (
    # 将第一次调用的结果放到参数synopsis中，再作为第二次review_prompt的输入
    {"synopsis": synopsis_prompt | llm | StrOutputParser()}
    | review_prompt
    | llm
    | StrOutputParser()
)
result = chain.invoke({"title": "日落时的海滩悲剧"})
print(result)

"日落时的海滩悲剧"是一部呼唤我们对人性和命运无常深入反思的剧作。剧本巧妙地通过主角杰克的个人经历，充满情感的描绘了对失去的爱和自我救赎的渴望，使观众深感共鸣。

故事的环境设置在美丽而神秘的海滩，这样的背景为本剧增添了一种独特的情感色彩。海滩，即是杰克与莉莉曾经的甜蜜乐园，也是杰克痛苦的源泉。这种强烈的对比使得剧情更加引人入胜。

杰克的角色塑造尤为出色。他的坚毅和勇敢，他对过去的痛苦回忆，以及他在寻求救赎的过程中的挣扎，都让人深感同情。而他在面对莉莉死亡真相时的决心和勇气，令人印象深刻。

在剧中出现的其他角色也丰富了剧情的层次感。神秘的海滩管家、曾目睹莉莉事故的渔夫，以及善良热情的艾米，她们的出现给杰克的生活带来了新的转机，也为观众提供了对主题的更多思考。

"日落时的海滩悲剧"的剧情独特，复杂而引人入胜。剧中的每一个情节都充满了戏剧张力，每一场戏都能触动人心。最重要的是，这部剧让我们看到了人性中的矛盾和复杂，也看到了在痛苦和希望中寻找救赎的可能。

总的来说，"日落时的海滩悲剧"是一部揭示人性深层次的剧作，充满了悲剧和希望。它不仅令人深思，也让人心生敬畏。我强烈推荐观众观看这部作品，去感受它带来的情感冲击和人性的反思。


In [6]:
#如果我们还想要回到概要，我们可以这样做
from langchain.schema.runnable import RunnablePassthrough

synopsis_chain = synopsis_prompt | llm | StrOutputParser()
review_chain = review_prompt | llm | StrOutputParser()
# 定义一个字典{"synopsis": synopsis_chain}；
# 再给到RunnablePassthrough.assign() 方法，将其与 review_chain 合并；
# RunnablePassthrough.assign()返回一个新的对象给chain
chain = {"synopsis": synopsis_chain} | RunnablePassthrough.assign(review=review_chain)
# 执行invoke时会自动将上一条链的输出，作为下一条链的输入
result = chain.invoke({"title": "日落时的海滩悲剧"})
print(result)


#RunnablePassthrough.assign 方法说明：
# 接受一个或多个键值对作为参数，其中键表示链路的名称，值表示链路本身。在这个例子中，
# 我们传入了两个键值对："synopsis": synopsis_chain 和 "review": review_chain。
# 当调用 RunnablePassthrough.assign() 方法时，它会创建一个新的 RunnablePassthrough 对象，
# 并将传入的链路添加到该对象中。这个新对象可以像字典一样使用，通过链路名称来访问和执行相应的链路

{'synopsis': '"日落时的海滩悲剧"是一部深情且引人深思的戏剧，描绘了在一个偏远海滩发生的一系列悲剧事件。故事围绕着两个家庭展开，他们在这个风景如画的海滩上过着平静的生活，直到一个可怕的秘密被揭露，改变了他们所有人的生活。\n\n海滩的主人，老渔夫约瑟夫，因为一个神秘的过去而寂寞地生活着。另一方面，刚搬到海滩的年轻家庭，由年轻的夫妇安娜和汤姆以及他们的小女儿莉莉组成，他们希望在这里开始新的生活。然而，当一个不速之客在日落时出现，揭示了约瑟夫的过去，这个宁静的海滩变成了一片混乱。\n\n这个侵入者是约瑟夫的旧知己，他带来了一个令人震惊的消息，约瑟夫的过去并不像他所描述的那样简单。他曾是一名海盗，被他的同伴背叛，被迫独自逃生。他的同伴为了掩盖真相，制造了一个关于约瑟夫的谎言。\n\n当这个秘密被揭露，海滩的宁静被打破，约瑟夫被迫面对他的过去，同时安娜和汤姆也被卷入了这场风暴。这部剧以一种深情而悲壮的方式描绘了人性的复杂性，以及面对过去和寻找救赎的艰难过程。', 'review': '《日落时的海滩悲剧》是一部引人深思的戏剧，它在揭示人性的微妙复杂性方面展现了出色的表现力。剧作家通过这部作品，巧妙地探讨了过去的阴影如何在未预料的瞬间破坏现在的安宁，以及人们如何在混乱中寻找救赎。\n\n首先，剧中的角色塑造极具深度。约瑟夫，这个老渔夫，被描绘成一个拥有神秘过去的人，他的寂寞和痛苦让人同情。另一方面，对新家庭的描述也充满了细腻的笔触，年轻的夫妇安娜和汤姆以及他们的小女儿莉莉，他们的希望和恐惧都让这个家庭显得栩栩如生。\n\n随着剧情的推进，不速之客的出现，揭示了约瑟夫的过去，这一点转折尤其出色。这个揭示不仅改变了角色们的生活，也改变了观众对约瑟夫的看法。他的过去并不像他所描述的那样简单，他曾是一名海盗，这种复杂的背景增加了剧情的张力，并为后续的冲突和纠葛做了铺垫。\n\n然而，本剧的真正亮点在于，它以一种深情而悲壮的方式描绘了人性的复杂性，以及面对过去和寻找救赎的艰难过程。约瑟夫被迫面对他的过去，安娜和汤姆也被卷入了这场风暴，他们的挣扎，他们的冲突，以及他们的最终救赎，都以一种深情而引人深思的方式呈现出来。\n\n总的来说，《日落时的海滩悲剧》是一部引人深思的戏剧，它以精彩的剧情和复杂的人物塑造展现了生活的真实性和人性的复杂性。对于喜欢深度剧情和复杂角色塑造的观众

## 3. Transformation

转换：数据传递过程中对其进行转换处理

例如，我们将创建一个虚拟转换，它接收一个超长的文本，将文本筛选为前三段，然后将其传递到一个链中进行摘要。

In [9]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """对下面内容进行总结摘要:
{output_text}
摘要:"""
)

In [10]:
with open("消失的她.txt",'r', encoding='utf-8') as f:
    state_of_the_union = f.read()

In [11]:
from langchain_openai import ChatOpenAI
from langchain.schema import StrOutputParser

llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125")

runnable = (
    {"output_text": lambda text: "\n\n".join(text.split("\n\n")[:3])}
    | prompt
    | llm
    | StrOutputParser()
)
result = runnable.invoke(state_of_the_union)
print(result)

电影《消失的她》讲述了丈夫何非的妻子李木子在结婚周年旅行中神秘失踪的故事。出现了一个陌生女人冒充李木子，并引发了一系列扑朔迷离的事件。情节复杂，存在矛盾和谜团，观众需猜测真相。


## 4. Router

RouterChain: 实现条件判断的大模型调用

路由允许创建非确定性链，其中前一步的输出定义下一步。路由有助于在与LLMs的交互中提供结构和一致性。

例子: 假设我们有两个针对不同类型问题进行优化的模板，并且我们希望根据用户输入选择模板。

In [12]:
from langchain.prompts import PromptTemplate

## 物理学家的模板
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{input}"""
physics_prompt = PromptTemplate.from_template(physics_template)

## 数学家的模板
math_template = """You are a very good mathematician. You are great at answering math questions. \
You are so good because you are able to break down hard problems into their component parts, \
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{input}"""
math_prompt = PromptTemplate.from_template(math_template)

In [13]:
from langchain_openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableBranch

## 其他通用问题的模块
general_prompt = PromptTemplate.from_template(
    "You are a helpful assistant. Answer the question as accurately as you can.\n\n{input}"
)

##路由分支
prompt_branch = RunnableBranch(
    (lambda x: x["topic"] == "math", math_prompt),
    (lambda x: x["topic"] == "physics", physics_prompt),
    general_prompt,
)

In [40]:
from typing import Literal

from langchain.pydantic_v1 import BaseModel
from langchain.output_parsers.openai_functions import PydanticAttrOutputFunctionsParser
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

## 对用户问题分类，定义函数
class TopicClassifier(BaseModel):
    "Classify the topic of the user question"

    topic: Literal["math", "physics", "general"]
    "The topic of the user question. One of 'math', 'physics' or 'general'."

## 转化为OpenAI function函数
classifier_function = convert_pydantic_to_openai_function(TopicClassifier)

## 创建大模型
llm = ChatOpenAI(model_name="gpt-3.5-turbo").bind(
    functions=[classifier_function], function_call={"name": "TopicClassifier"}
)
## 定义对输出进行解析
## 输出的对象TopicClassifier的属性值topic
parser = PydanticAttrOutputFunctionsParser(
    pydantic_schema=TopicClassifier, attr_name="topic"
)
## 基于大模型对输出进行解析
classifier_chain = llm | parser

In [41]:
from operator import itemgetter

from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough


final_chain = (
    RunnablePassthrough.assign(topic=itemgetter("input") | classifier_chain)
    | prompt_branch
    | ChatOpenAI(model_name="gpt-3.5-turbo-0125")
    | StrOutputParser()
)

In [42]:
## 什么是大于40的第一个质数，使得这个质数加一可被3整除？
result = final_chain.invoke(
    {
        "input": "What is the first prime number greater than 40 such that one plus the prime number is divisible by 3?"
    }
)

print(result)

"Thank you for your kind words! I'm happy to help you with this math question.\n\nTo find the first prime number greater than 40 such that one plus the prime number is divisible by 3, we can follow these steps:\n\n1. Start by listing the prime numbers greater than 40. The prime numbers after 40 are 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, and so on.\n\n2. For each prime number, add one to it and check if the resulting number is divisible by 3.\n\nLet's go through the list:\n\n- For 41, one plus 41 equals 42, which is divisible by 3. However, 41 itself is not divisible by 3.\n- For 43, one plus 43 equals 44, which is not divisible by 3.\n- For 47, one plus 47 equals 48, which is divisible by 3. However, 47 itself is not divisible by 3.\n\nWe can continue this process until we find the first prime number that satisfies the condition. \n\nLet's check the next prime number:\n\n- For 53, one plus 53 equals 54, which is divisible by 3. Therefore, the first prime number greater tha